In [4]:
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
import glob


In [5]:
# To determine the right zones that belong to Manhattan
zones = pd.read_csv('taxi_zone_lookup.csv')
zones['Borough'] == 'Manhattan'
manhattan_zones = zones[zones['Borough'] == 'Manhattan']
manhattan_zone_numbers = manhattan_zones['LocationID'].unique().tolist()
print(manhattan_zone_numbers)

[4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 103, 104, 105, 107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 153, 158, 161, 162, 163, 164, 166, 170, 186, 194, 202, 209, 211, 224, 229, 230, 231, 232, 233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]


In [8]:
# Looped version for makeing the datasets
years = list(range(2012, 2025))

# Loop through each year and process the Parquet file
for year in years:
    file_path = f'yellow_tripdata_{year}-07.parquet'
    
    # Read Parquet file
    taxi_data = pq.read_table(file_path).to_pandas()

    # Select necessary columns
    taxi_data = taxi_data[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID']]

    # Convert datetime column to proper format
    taxi_data['tpep_pickup_datetime'] = pd.to_datetime(taxi_data['tpep_pickup_datetime'])
    taxi_data['tpep_dropoff_datetime'] = pd.to_datetime(taxi_data['tpep_dropoff_datetime'])

    # Separate Date and Time Columns
    taxi_data['pickup_date'] = taxi_data['tpep_pickup_datetime'].dt.date
    taxi_data['pickup_time'] = taxi_data['tpep_pickup_datetime'].dt.time
    taxi_data['dropoff_date'] = taxi_data['tpep_dropoff_datetime'].dt.date
    taxi_data['dropoff_time'] = taxi_data['tpep_dropoff_datetime'].dt.time
    
    # Filter for Manhattan zones
    taxi_data = taxi_data[
        (taxi_data['PULocationID'].isin(manhattan_zone_numbers)) & 
        (taxi_data['DOLocationID'].isin(manhattan_zone_numbers))
    ]
    # Filter for the first week of July (July 1–7, 2011)
    start_date = f'{year}-07-01'
    end_date = f'{year}-07-07'
    
    taxi_data = taxi_data[
        (taxi_data['pickup_date'] >= pd.to_datetime(start_date).date()) & 
        (taxi_data['dropoff_date'] <= pd.to_datetime(end_date).date())
    ]
     # Sort data by pickup date & pickup time
    taxi_data = taxi_data.sort_values(by=['pickup_date', 'pickup_time'])

    taxi_data['trip_length'] = taxi_data['tpep_dropoff_datetime'] - taxi_data['tpep_pickup_datetime']
    taxi_data['length_trip_min'] = taxi_data['trip_length'].dt.total_seconds() / 60
    taxi_data = taxi_data[taxi_data['length_trip_min'] > 0]
    taxi_data = taxi_data.round({'length_trip_min':2})
    taxi_data = taxi_data[['pickup_date','pickup_time', 'dropoff_date', 'dropoff_time', 'length_trip_min', 'PULocationID', 'DOLocationID']]
   
   
    # Save filtered data as CSV
    output_filename = f'{year}_new.parquet'
    taxi_data.to_parquet(output_filename, index=False)

    print(f"Processed and saved: {output_filename}")
    # print(taxi_data_filtered)
    
# print(taxi_data[taxi_data['length_trip_min']<=0]['length_trip_min'])
# taxi_data

Processed and saved: 2012_new.parquet
Processed and saved: 2013_new.parquet
Processed and saved: 2014_new.parquet
Processed and saved: 2015_new.parquet
Processed and saved: 2016_new.parquet
Processed and saved: 2017_new.parquet
Processed and saved: 2018_new.parquet
Processed and saved: 2019_new.parquet
Processed and saved: 2020_new.parquet
Processed and saved: 2021_new.parquet
Processed and saved: 2022_new.parquet
Processed and saved: 2023_new.parquet
Processed and saved: 2024_new.parquet


In [ ]:
 # taxi_data_filtered_week1['Pickup_FullDateTime'] = pd.to_datetime(taxi_data_filtered_week1['pickup_date'].astype(str) + ' ' + taxi_data_filtered_week1['pickup_time'].astype(str))
    # taxi_data_filtered_week1['Dropoff_FullDateTime'] = pd.to_datetime(taxi_data_filtered_week1['dropoff_date'].astype(str) + ' ' + taxi_data_filtered_week1['dropoff_time'].astype(str))
    
    # # Compute trip length as a timedelta
    # taxi_data_filtered_week1 ['length_trip'] = taxi_data_filtered_week1 ['Dropoff_FullDateTime'] - taxi_data_filtered_week1 ['Pickup_FullDateTime']
    
    # # Optional: Convert to minutes or seconds
    # taxi_data_filtered_week1['length_trip_minutes'] = taxi_data_filtered_week1 ['length_trip'].dt.total_seconds() / 60
    
    # # Drop the full datetime columns if not needed
    # # taxi_2010_filterd_week1 = taxi_2010_filterd_week1.drop(columns=['Pickup_FullDateTime', 'Dropoff_FullDateTime'])
    
    # # Display results
    # # print(taxi_data_filtered_week1[['pickup_time', 'dropoff_time', 'length_trip', 'length_trip_minutes']])
    # taxi_data_filtered_week1 = taxi_data_filtered_week1[taxi_data_filtered_week1['length_trip_minutes'] > 0]
    # taxi_data = taxi_data[['pickup_date','pickup_time', 'dropoff_date', 'dropoff_time', 'length_trip_minutes', 'PULocationID', 'DOLocationID']]
    
    # # test all all fault test are out
    # print(taxi_data_filtered_week1[taxi_data_filtered_week1['length_trip_minutes']<=0]['length_trip_minutes'])

In [ ]:
# taxi_2011 = pq.read_table('yellow_tripdata_2011-07.parquet')
# taxi_2011 = taxi_2011.to_pandas()

# # Select necessary columns
# taxi_2011 = taxi_2011[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID']]

# # Convert datetime column to proper format
# taxi_2011['tpep_pickup_datetime'] = pd.to_datetime(taxi_2011['tpep_pickup_datetime'])
# taxi_2011['tpep_dropoff_datetime'] = pd.to_datetime(taxi_2011['tpep_dropoff_datetime'])

# # Filter for Manhattan zones
# taxi_filtered_2011 = taxi_2011[
#     (taxi_2011['PULocationID'].isin(manhattan_zone_numbers)) & 
#     (taxi_2011['DOLocationID'].isin(manhattan_zone_numbers))
# ]

# # Filter for the first week of July (July 1–7, 2011)
# start_date = '2011-07-01'
# end_date = '2011-07-07'

# taxi_filtered_2011_week1 = taxi_filtered_2011[
#     (taxi_filtered_2011['tpep_pickup_datetime'] >= start_date) & 
#     (taxi_filtered_2011['tpep_pickup_datetime'] <= end_date)
# ]

In [ ]:
#     # Save filtered data as CSV
# output_filename = f'yellow_tripdata_2011-07_filtered.csv'
# taxi_filtered_2011.to_csv(output_filename, index=False)

# print(f"Processed and saved:" )